In [1]:
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

## Data

In [3]:
acs = pd.read_csv('../modeling/final_acs_transportation_choice.csv')#, index_col=0)
acs.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
0,3.0,0.115434,17.843262,7.334361,3885.402712,478.622467,4316.681764
1,4.0,42.851015,140.891180,84.609811,6937.780033,2631.714648,779.153313
2,5.0,0.081377,13.158607,2.120444,1860.706347,40.495673,6312.437553
3,6.0,0.109017,7.637848,1.878344,1974.772111,198.118892,3086.483787
4,7.0,25.522468,142.614028,37.582529,28436.765508,2680.397880,5365.117587


In [4]:
mode_data = pd.read_csv('../modeling/final_allMode_with_2017wage_cleaned_update.csv', index_col=0)
mode_data = mode_data.dropna()
mode_data.head()

,DOlocationID,ODpair,PUlocationID,duration,mode,nest,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000
0,4,3-4,3,39.695000,2,1,64.000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
1,4,3-4,3,45.216667,3,1,61.500,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
2,4,3-4,3,83.000000,4,2,5.500,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
3,4,3-4,3,225.933333,5,3,0.000,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005
4,4,3-4,3,39.695000,6,4,29.424,0.250819,0.360216,0.648452,0.282211,0.414755,0.592009,0.671282,0.93859,0.586936,0.740725,0.514005


### Apply to our case

In [23]:
def compare_with_ground_truth(predictdf, truedf):
    '''
    compare our predicted transportation choice with ground truth
    
    The header of the datafrme after merge (named 'data_compare') should be like:
    taxi_zone | P(mode1)_x | P(mode2)_x | P(mode3)_x | P(mode4)_x | P(mode5)_x | P(mode1)_y | P(mode2)_y | P(mode3)_y | P(mode4)_y | P(mode5)_y
    '''
    import numpy as np
    # makesure predictdf and truedf have the same formats
    for col in predictdf.columns:
        predictdf[col] = predictdf[col].astype(float) 
    predictdf = predictdf.fillna(0)
    predictdf = predictdf.replace([np.inf, -np.inf], np.nan)
    predictdf = predictdf.dropna()
    predictdf['taxi_zone'] = predictdf.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    predictdf = predictdf.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    predictdf['taxi_zone'] = predictdf['taxi_zone'].astype(int)
    
    #do same thing as predictdf to loss_function_deno dataframe
#     for col in loss_function_deno.columns:
#         loss_function_deno[col] = loss_function_deno[col].astype(float) 
#     loss_function_deno = loss_function_deno.fillna(0)
#     loss_function_deno = loss_function_deno.replace([np.inf, -np.inf], np.nan)
#     loss_function_deno = loss_function_deno.dropna()
#     loss_function_deno['taxi_zone'] = loss_function_deno.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
#     loss_function_deno = loss_function_deno.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
#     loss_function_deno['taxi_zone'] = loss_function_deno['taxi_zone'].astype(int)   
    
    truedf['taxi_zone'] = truedf['taxi_zone'].astype(int)
    
    data_compare = pd.merge(predictdf, truedf, left_on='taxi_zone', right_on = 'taxi_zone')

    data_compare = data_compare.dropna() 
    data_compare_zone = data_compare['taxi_zone'].unique()
#     loss_function_deno = loss_function_deno[loss_function_deno['taxi_zone'].isin(data_compare_zone)]

    rloss = 0
    rmse = 0
    for i in range(1,totmode+1):
        data_compare_deno = (data_compare[data_compare.columns[i+totmode]].replace(to_replace=0,value = 1)).copy()
        rlossi = (np.sqrt(sum(((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+totmode]])**2)\
                        /(data_compare_deno))))
        rmsei = (np.sqrt(sum((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+totmode]])**2)))
        if rlossi == np.nan: print('Nan encountered')
    rloss += rlossi
    rmse += rmsei
    tot_mse = np.sqrt(sum((np.asarray(predictdf.sum()[1:]) - np.asarray(truedf.sum()[1:]))**2))
    return rloss, rmse, tot_mse

In [24]:
totmode = 6

1

In [207]:
predict_choice_1 = pd.read_csv('Final_results/results_scenario3_with_scenario1_0.04_1.5.csv',\
                               index_col=0)

In [208]:
rloss1, rmse1, tot_mse1 = compare_with_ground_truth(predict_choice_1, acs)

In [209]:
predict_choice_1.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
3-4,0.0,0.0,0.0,4.492907,0.249570,1.257523
3-7,0.0,0.0,0.0,12.012387,1.458571,22.529042
3-9,0.0,0.0,0.0,0.140546,0.072559,1.786895
3-10,0.0,0.0,0.0,0.157064,0.124834,2.718103
3-11,0.0,0.0,0.0,3.853464,0.145975,0.000561


2

In [212]:
predict_choice_2 = pd.read_csv('Final_results/results_scenario3_with_scenario1_0.05_1.5.csv',index_col=0)

In [213]:
rloss2, rmse2, tot_mse2 = compare_with_ground_truth(predict_choice_2, acs)

3

In [214]:
predict_choice_3 = pd.read_csv('Final_results/results_scenario3_with_scenario1_0.06_1.5.csv',index_col=0)

In [215]:
rloss3, rmse3, tot_mse3 = compare_with_ground_truth(predict_choice_3, acs)

4

In [216]:
predict_choice_4 = pd.read_csv('Final_results/results_scenario3_with_scenario1_0.05_1.4.csv',index_col=0)

In [217]:
rloss4, rmse4, tot_mse4 = compare_with_ground_truth(predict_choice_4, acs)

5

In [218]:
predict_choice_5 = pd.read_csv('Final_results/results_scenario3_0.05_1.6.csv',index_col=0)

In [219]:
rloss5, rmse5, tot_mse5 = compare_with_ground_truth(predict_choice_5, acs)

In [220]:
predict_choice_5.shape

(55798, 6)

In [191]:
predict_choice_1['rmse'] = rmse1
predict_choice_2['rmse'] = rmse2
predict_choice_3['rmse'] = rmse3
predict_choice_4['rmse'] = rmse4
predict_choice_5['rmse'] = rmse5

In [192]:
combined = predict_choice_1.append(predict_choice_2.append(predict_choice_3.append\
                                                           (predict_choice_4.append(predict_choice_5))))

In [193]:
# wm = lambda dfx: (dfx * np.exp(-dfx["rmse"])).sum() / np.exp(-dfx["rmse"]).sum()

In [194]:
wm = lambda x: pd.Series(np.average(x, weights= (x['rmse']),axis=0))

In [195]:
mean_combined = combined.groupby(combined.index).apply(wm)

In [196]:
mean_combined.columns = predict_choice_1.columns

In [197]:
mean_combined.drop(['rmse'],axis=1,inplace=True)

In [210]:
mean_combined.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
10-100,0.000000e+00,0.000000e+00,0.000000e+00,131.060043,4.939957,3.994706e-08
10-101,0.000000e+00,3.753789e-28,9.020862e-16,1.942583,18.875054,7.118236e+01
10-102,8.402175e-09,3.583028e-65,7.742238e-57,0.042107,5.670483,4.328741e+01
10-106,0.000000e+00,0.000000e+00,0.000000e+00,5.242865,2.022793,2.073434e+01
10-107,0.000000e+00,0.000000e+00,0.000000e+00,39.001538,1.200242,6.798220e+00


In [198]:
mean_combined.shape

(55798, 6)

In [199]:
mean_combined.dropna().shape

(55798, 6)

In [200]:
def weighted_std(values):
    average = np.average(values, weights=values['rmse'],axis=0)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=values['rmse'],axis=0)
    return (pd.Series(np.sqrt(variance)))

In [201]:
std_combined = combined.groupby(combined.index).apply(weighted_std)

In [202]:
std_combined.columns = predict_choice_1.columns

In [203]:
std_combined.drop(['rmse'],axis=1,inplace=True)

In [211]:
std_combined.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
10-100,0.000000e+00,0.000000e+00,0.000000e+00,0.055677,0.055678,3.468281e-08
10-101,0.000000e+00,7.554154e-28,1.815365e-15,0.110739,0.089100,2.169308e-02
10-102,1.070189e-08,7.210512e-65,1.558054e-56,0.007985,0.025502,3.344052e-02
10-106,0.000000e+00,0.000000e+00,0.000000e+00,0.007415,0.004048,3.366640e-03
10-107,0.000000e+00,0.000000e+00,0.000000e+00,0.029117,0.000804,2.991931e-02


In [204]:
std_combined.shape

(55798, 6)

In [205]:
std_combined.dropna().shape

(55798, 6)

In [206]:
mean_combined.to_csv('Final_results/Uncertainty/mean_scenario_2.csv')
std_combined.to_csv('Final_results/Uncertainty/std_scenario_2.csv')